In [25]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import timeit
import time
from sklearn.utils import shuffle
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

<h1>Extract MNIST data</h1>

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding, reshape = False (that means images are not flatten)
mnist = input_data.read_data_sets("MNIST_data/",reshape=False,one_hot=True)
#suppress warnings
tf.logging.set_verbosity(old_v)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h1>Prepare training, validation and testing data</h1>

In [12]:
x_train, y_train           = mnist.train.images, mnist.train.labels
x_validation, y_validation = mnist.validation.images, mnist.validation.labels
x_test, y_test             = mnist.test.images, mnist.test.labels

x_train_reshape = []
x_validation_reshape = []
x_test_reshape = []
#pad images with 0s (28x28 to 32x32)
for i in range(len(x_train)):
    pad = np.pad(x_train[i], ((2,2),(2,2),(0,0)), 'constant')
    x_train_reshape.append(pad)
for i in range(len(x_validation)):
    pad = np.pad(x_validation[i], ((2,2),(2,2),(0,0)), 'constant')
    x_validation_reshape.append(pad)
for i in range(len(x_test)):
    pad = np.pad(x_test[i], ((2,2),(2,2),(0,0)), 'constant')
    x_test_reshape.append(pad)
print(x_test_reshape[0].shape)

(32, 32, 1)


In [11]:
print(y_train[0].shape)

(10,)


<h1>Define hyperparameter</h1>

In [90]:
batch_size = 256
lr = 0.0008
epoch = 30
inp_size = 32
conv1 = {"size": 5, "nums": 6, "stride": 1}
conv2 = {"size": 5, "nums": 16, "stride": 1}
maxPool = {"size": 2, "stride": 2}
hidden_size1 = 120
hidden_size2 = 84
out_size = 10

In [91]:
pass_num = len(x_train) // batch_size
def get_batch(i, size):
    inp = [inp for inp in x_train_reshape[(i * size):((i + 1) * size)]]
    label = [lab for lab in y_train[(i * size):((i + 1) * size)]]
    X_inp = np.empty((len(inp), inp_size, inp_size, 1), dtype=np.float32)
    y_label = np.empty((len(inp),10), dtype=np.float32)
    for j in range(len(inp)):
        X_inp[j] = inp[j]
        y_label[j] = label[j]
    return (X_inp, y_label)

def val_data():
    X_inp = np.empty((len(x_validation_reshape), inp_size, inp_size, 1), dtype=np.float32)
    y_label = np.empty((len(x_validation_reshape), 10), dtype=np.float32)
    for j in range(len(x_validation_reshape)):
        X_inp[j] = x_validation_reshape[j]
        y_label[j] = y_validation[j]
    return (X_inp, y_label)
val_x, val_y = val_data()
print(val_x.shape, val_y.shape)

def get_batch_test(i, size):
    inp = [inp for inp in x_test_reshape[(i * size):((i + 1) * size)]]
    label = [lab for lab in y_test[(i * size):((i + 1) * size)]]
    X_inp = np.empty((len(inp), inp_size, inp_size, 1), dtype=np.float32)
    y_label = np.empty((len(inp),10), dtype=np.float32)
    for j in range(len(inp)):
        X_inp[j] = inp[j]
        y_label[j] = label[j]
    return (X_inp, y_label)

(5000, 32, 32, 1) (5000, 10)


In [92]:
tf.reset_default_graph()

<h1>Placeholder</h1>

In [93]:
with tf.name_scope("data"):
    X = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 1])
    y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

<h1>Define LeNet-5</h1>

In [94]:
def LeNet5(X):
    layer1_out = tf.layers.conv2d(X, conv1["nums"], conv1["size"], conv1["stride"], 'valid', activation=tf.nn.relu)
    maxPool1_out = tf.contrib.layers.max_pool2d(layer1_out, maxPool["size"], maxPool["stride"], 'VALID')
    
    layer2_out = tf.layers.conv2d(maxPool1_out, conv2["nums"], conv2["size"], conv2["stride"], 'valid', activation=tf.nn.relu)
    maxPool2_out = tf.contrib.layers.max_pool2d(layer2_out, maxPool["size"], maxPool["stride"], 'VALID')
    
    layer2_reshape = tf.reshape(maxPool2_out, shape=[-1, conv2["nums"] * 5 * 5])
    fc1 = tf.layers.dense(layer2_reshape, hidden_size1, activation=tf.nn.relu, use_bias=True)
    fc2 =  tf.layers.dense(fc1, hidden_size2, activation=tf.nn.relu, use_bias=True)
    out = tf.layers.dense(fc2, out_size, activation=tf.nn.softmax, use_bias=True)
    return out
out = LeNet5(X)

<h1>Cost and optimization</h1>

In [95]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=out)
correct_pred = tf.equal(tf.argmax(out,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()

<h1>Training, validating, testing</h1>
<h2>1. Print out validation accuracy after each training epoch</h2>
<h2>2. Print out training time on each epoch</h2>
<h2>3. Print out testing accuracy</h2>

In [96]:
start = time.time()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for i in range(epoch):
        for j in range(pass_num):
            batch_x, batch_y = get_batch(j, batch_size)
            sess.run(train_op, feed_dict={X:batch_x, y:batch_y})
            if j == pass_num - 1:
                tr_acc = sess.run(accuracy,feed_dict={X:batch_x, y:batch_y})
                print("epoch "+str(i)+", Training accuracy= {:.3f}".format(tr_acc))
        val_acc = sess.run(accuracy,feed_dict={X:val_x, y:val_y})
        print("epoch "+str(i)+", Validation accuracy= {:.3f}".format(val_acc))
        print("time elapsed at "+str(i)+"th epoch is: ", int(time.time() - start), " seconds")
    save_path = saver.save(sess, 'model.ckpt')
    print("Training finished!")
    acc_test = 0.0
    count = 0
    pass_num = len(y_test) // batch_size
    for k in range(pass_num):
            batch_x, batch_y = get_batch_test(k, batch_size)
            test_acc = sess.run(accuracy, feed_dict={X:batch_x, y:batch_y})
            acc_test += test_acc
            count += 1
    print("Test accuracy is: {:.3f}".format(acc_test / count))

epoch 0, Training accuracy= 0.703
epoch 0, Validation accuracy= 0.695
time elapsed at 0th epoch is:  1  seconds
epoch 1, Training accuracy= 0.887
epoch 1, Validation accuracy= 0.885
time elapsed at 1th epoch is:  3  seconds
epoch 2, Training accuracy= 0.895
epoch 2, Validation accuracy= 0.893
time elapsed at 2th epoch is:  4  seconds
epoch 3, Training accuracy= 0.961
epoch 3, Validation accuracy= 0.976
time elapsed at 3th epoch is:  6  seconds
epoch 4, Training accuracy= 0.969
epoch 4, Validation accuracy= 0.977
time elapsed at 4th epoch is:  8  seconds
epoch 5, Training accuracy= 0.969
epoch 5, Validation accuracy= 0.981
time elapsed at 5th epoch is:  9  seconds
epoch 6, Training accuracy= 0.969
epoch 6, Validation accuracy= 0.981
time elapsed at 6th epoch is:  11  seconds
epoch 7, Training accuracy= 0.969
epoch 7, Validation accuracy= 0.982
time elapsed at 7th epoch is:  12  seconds
epoch 8, Training accuracy= 0.969
epoch 8, Validation accuracy= 0.986
time elapsed at 8th epoch is:  1